Para el correcto funcionamiento de este fichero es necesario crear dentro de Colab las carpetas ods, csv, txt y pdf. Dentro de las carpetas pdf y ods deberán estar todos los ficheros a transformar

In [1]:
import pandas as pd
!pip install tika
from tika import parser
from google.colab import files
!pip install odfpy

  Preparing metadata (setup.py) ... done
  Created wheel for tika: filename=tika-2.6.0-py3-none-any.whl size=32625 sha256=eed47d50b963cf9efc6e941d8651fbbc51703e9cdba25cc1cfdc4ccf9473a524
  Stored in directory: /root/.cache/pip/wheels/5f/71/c7/b757709531121b1700cffda5b6b0d4aad095fb507ec84316d0
Successfully built tika
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 717.0/717.0 kB 4.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for odfpy: filename=odfpy-1.4.1-py2.py3-none-any.whl size=160672 sha256=f366f6b3fae34e76ddf952942bea7030991cc8d6ac9409145857f356d03f1e95
  Stored in directory: /root/.cache/pip/wheels/c8/2e/95/90d94fe33903786937f3b8c33dd88807f792359c6424b40469
Successfully built odfpy


In [2]:
def init():

  # opening pdf file
  parsed_pdf = parser.from_file(ruta + "pdf/" + document + ".pdf")

  # saving content of pdf
  # you can also bring text only, by parsed_pdf['text']
  # parsed_pdf['content'] returns string
  data = parsed_pdf['content']

  # writing to file
  with open(ruta + "txt/" + document + ".txt", 'w') as f:
      f.write(data)

  # reading from file
  file1 = open(ruta + "txt/" + document + ".txt", 'r')
  Lines = file1.readlines()
  extractPages(pagina, pages, Lines)

In [3]:
# Crea listas de cada página
def extractPages(myList, pages, Lines):
  for line in Lines:
    if line.find("de Alicante") != -1:
      pages +=1
      if pages !=0:
        extractDataModificados(myList, pages)
        myList = []
    if line != '\n' and line != ' \n':
      myList.append(line)
  extractDataModificados(myList, pages+1)

In [4]:
# Extrae los datos de una página de los contratos modificados guardada en una lista de Strings
def extractDataModificados(pagina, pages):

  # Inicializamos las variables para poder usarlas en cada llamada e introducirlas en el DataSet tras el bucle For

  numExpediente = objContrato = organismo  = tipoContrato = tipoTramitacion = " "
  fechaAprobacion = fechaInicio = importe = causa = linea2 =  " "
  idObjContrato = idCausa = -1

  # print("Aqui se extraerían los datos de la página " + str(pages))
  for count, value in enumerate(pagina):
    if "Nº Expediente:" in value:
      numExpediente = value[(value.find("Nº Expediente:")) + 15:]
    if "Objeto del contrato:" in value:
      idObjContrato = count - 1
      while ("Presupuesto de licitación:" not in pagina[idObjContrato]):
        idObjContrato -= 1
      for x in range(idObjContrato+1, count):
        linea2 = pagina[x]
        objContrato += linea2
      objContrato = objContrato.replace('\n', " ")
    if "Órgano gestor:" in value:
      organismo = value[(value.find("Órgano gestor:") + 15):]
    if "Nº Registro:" in value:
      tipoContrato = pagina[count + 1]
    if "Fecha formalización:" in value:
      tipoTramitacion = pagina[count - 1]
      fechaAprobacion = value[(value.find("garantía:") + 9): (value.find("garantía:") + 20)]
      fechaInicio = value[(value.find("garantía:") + 20):]
    if "Variación precio:" in value:
      importe = value[(value.find("Variación precio: ") + 18):]
    if "Causas de la modificación:" in value:
      # if "Variación precio:" not in pagina[count +1]:
      #   causa = pagina[count + 1]
      idCausa = count + 1
      while ("Variación precio:" not in pagina[idCausa]):
        linea2 = pagina[idCausa]
        idCausa += 1
        causa += linea2
      causa = causa.replace('\n', " ")

  # Creamos la lista fila con todos los datos de una fila del DataSet e introducimos la fila en el fichero, esto se repite en los x llamadas que se hacen a la función,
  # x equivalente al número de páginas del documento
  fila = [numExpediente, objContrato, organismo, tipoContrato, tipoTramitacion, fechaAprobacion, fechaInicio, importe, causa]
  modificados.loc[len(modificados)] = fila
  # modificados.to_csv(ruta + "csv/modificados_" + str(anyo) + ".csv")
  modificados.to_csv(ruta + "csv/modificados_" + str(anyo) + ".csv", encoding='utf-8')


In [5]:
# Convierte los archivos ODS a CSV para poder juntar todos en el mismo formato
def ods_to_csv():
  for anyo in range (2019, 2023):
    # print(str(anyo))
    columnas = ["Num. Exp",
              "Objeto Contrato",
              "Organismo",
              "Tipo de Contrato",
              "Tipo Tramitación",
              "Fecha Aprobación",
              "Fecha Inicio",
              "Importe",
              "Causa/Justificación"]
    columnasODS = [ 'Num. Exp',
                    'Organismo',
                    'Objeto Contrato',
                    "Tipo de Contrato",
                    "Tipo Tramitación",
                    "Importe",
                    "Fecha Aprobación",
                    "Fecha Inicio",
                    "Causa/Justificación"]

    ruta = "/content/"
    document = "ods/modificados_" + str(anyo) + ".ods"

    if anyo == 2019:
      modificados = pd.read_excel(ruta + document, engine="odf", header=2)
      modificados.columns = columnasODS
      modificados=modificados.reindex(columns=columnas)
    if anyo == 2020:
      modificados = pd.read_excel(ruta + document, engine="odf", header=1)
      modificados.columns = columnasODS
      modificados=modificados.reindex(columns=columnas)
    if anyo == 2021:
      modificados = pd.read_excel(ruta + document, engine="odf", header=4)
      modificados = modificados.drop(columns="Unnamed: 2")
      modificados = modificados.drop(columns="Tipo Incidencia")
      columnasODS = ['Organismo',
                    'Num. Exp',
                    'Objeto Contrato',
                    "Tipo de Contrato",
                    "Tipo Tramitación",
                    "Importe",
                    "Fecha Aprobación",
                    "Fecha Inicio",
                    "Causa/Justificación"]
      modificados.columns = columnasODS
      modificados=modificados.reindex(columns=columnas)
    if anyo == 2022:
      modificados = pd.read_excel(ruta + document, engine="odf", header=3)
      modificados = modificados.drop(0)
      modificados = modificados.drop(columns="Unnamed: 0")
      modificados.insert(0, "Organismo", "NaN")
      modificados.insert(0, "Tipo Tramitación", "NaN")
      columnasODS = ["Tipo Tramitación",
                    "Organismo",
                    "Num. Exp",
                    "Objeto Contrato",
                    "Tipo de Contrato",
                    "Fecha Aprobación",
                    "Fecha Inicio",
                    "Importe",
                    "Causa/Justificación"]
      modificados.columns = columnasODS
      modificados=modificados.reindex(columns=columnas)

    # modificados.to_csv(ruta + "csv/modificados_" + str(anyo) + ".csv")
    modificados.to_csv(ruta + "csv/modificados_" + str(anyo) + ".csv", encoding='utf-8')

In [6]:
# Une los contratos de todos los años, añadiendo una columna "Año" en un archivo CSV
def unir_modificados():
  union = pd.DataFrame()
  for anyo in range (2015, 2023):
    ruta = "/content/"
    document = "csv/modificados_" + str(anyo) + ".csv"

    modificados = pd.read_csv(ruta+document)
    modificados = modificados.drop(columns="Unnamed: 0")

    modificados.insert(len(columnas), "Año", anyo)
    union = pd.concat([union, modificados])


    # display(union)
  union.to_csv(ruta + "modificados" + ".csv")
  union.to_csv(ruta + "modificados" + ".csv", encoding='utf-8')

In [7]:
# Ejecutar todo
for anyo in range (2015, 2021):
  pagina = []
  pages = -1
  columnas = ["Num. Exp",
              "Objeto Contrato",
              "Organismo",
              "Tipo de Contrato",
              "Tipo Tramitación",
              "Fecha Aprobación",
              "Fecha Inicio",
              "Importe",
              "Causa/Justificación"]
  modificados = pd.DataFrame(columns = columnas)
  ruta = "/content/"
  document = "contratos-modificados-" + str(anyo);
  init()

ods_to_csv()

2023-06-29 07:50:08,800 [MainThread  ] [INFO ]  Retrieving http://search.maven.org/remotecontent?filepath=org/apache/tika/tika-server-standard/2.6.0/tika-server-standard-2.6.0.jar to /tmp/tika-server.jar.
INFO:tika.tika:Retrieving http://search.maven.org/remotecontent?filepath=org/apache/tika/tika-server-standard/2.6.0/tika-server-standard-2.6.0.jar to /tmp/tika-server.jar.
2023-06-29 07:50:09,328 [MainThread  ] [INFO ]  Retrieving http://search.maven.org/remotecontent?filepath=org/apache/tika/tika-server-standard/2.6.0/tika-server-standard-2.6.0.jar.md5 to /tmp/tika-server.jar.md5.
INFO:tika.tika:Retrieving http://search.maven.org/remotecontent?filepath=org/apache/tika/tika-server-standard/2.6.0/tika-server-standard-2.6.0.jar.md5 to /tmp/tika-server.jar.md5.
2023-06-29 07:50:09,670 [MainThread  ] [WARNI]  Failed to see startup log message; retrying...
2023-06-29 07:50:14,685 [MainThread  ] [WARNI]  Failed to see startup log message; retrying...


In [8]:
unir_modificados()

In [9]:
for x in range (2015, 2023):
  files.download("/content/csv/modificados_" + str(x) + ".csv")
files.download("/content/modificados" + ".csv")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

...

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>